In [3]:
import json
import numpy as np
import os
from pprint import pprint

### RACE: Unfiltered Workers
# prompt_type, task_dir = 'question', '/Users/ethanperez/research/ParlAI/parlai/mturk/core/run_data/live/context_evaluator_1553982706'  # Q-only
# prompt_type, task_dir = 'context_question', '/Users/ethanperez/research/ParlAI/parlai/mturk/core/run_data/live/context_evaluator_1553790696'  # TFIDF
# prompt_type, task_dir = 'context_question', '/Users/ethanperez/research/ParlAI/parlai/mturk/core/run_data/live/context_evaluator_1553901953'  # FastText
### RACE: Filtered Workers
# prompt_type, task_dir = 'question', '/Users/ethanperez/research/ParlAI/parlai/mturk/core/run_data/live/context_evaluator_1554052233'  # Q-only
# prompt_type, task_dir = 'quote and question', '/Users/ethanperez/research/ParlAI/parlai/mturk/core/run_data/live/context_evaluator_1554006689'  # TFIDF(Q+O)
# prompt_type, task_dir = 'quote and question', '/Users/ethanperez/research/ParlAI/parlai/mturk/core/run_data/live/context_evaluator_1554130485'  # TFIDF(O)
# prompt_type, task_dir = 'quote and question', '/Users/ethanperez/research/ParlAI/parlai/mturk/core/run_data/live/context_evaluator_1554069931'  # Oracle
# prompt_type, task_dir = 'quote and question', '/Users/ethanperez/research/ParlAI/parlai/mturk/core/run_data/live/context_evaluator_1554072277'  # SL
# prompt_type, task_dir = 'quote and question', '/Users/ethanperez/research/ParlAI/parlai/mturk/core/run_data/live/context_evaluator_1554132868'  # SL-Influence

### DREAM
prompt_type, task_dir = 'question', '/Users/ethanperez/research/ParlAI/parlai/mturk/core/run_data/live/context_evaluator_1554582693'  # Q-only

# split = 'middle'
split = None

# Set useful variables
num_options = 4
options = ['A', 'B', 'C', 'D'][:num_options]
debate_mode_to_option = {'Ⅰ': 'A', 'Ⅱ': 'B', 'Ⅲ': 'C', 'Ⅳ': 'D', 'ⅰ': 'A', 'ⅱ': 'B', 'ⅲ': 'C', 'ⅳ': 'D', None: None}

# Read HIT data
print('# HIT Files:', len(os.listdir(task_dir)))
hit_results = []
num_passed_test = 0
for hit_dir in os.listdir(task_dir):
    if hit_dir.startswith('o_'):
        continue
    num_passed_test += 1
    with open(os.path.join(task_dir, hit_dir, 'custom/data.json'), 'r') as file:
        hit_results.append(json.load(file))
        file.close()
if len(os.listdir(task_dir)) != num_passed_test:
    num_total_tested = len(os.listdir(task_dir)) - num_passed_test
    print('# Passed Test:', num_passed_test, '/', num_total_tested, '=', round((100. * num_passed_test) / num_total_tested, 2), '%')

# HIT Files: 81
# Passed Test: 32 / 49 = 65.31 %


In [4]:
num_valid_hits, num_rejected_hits, num_incomplete_hits = 0, 0, 0
metrics = {}
task_ratings = {i: 0 for i in range(11)}
durations = []
durations_by_worker = []
accuracy_by_worker = {}
max_response_freqs = []
worker_ids = []

for hit_result in hit_results:
    if ((len(hit_result['reject_reasons']) > 0) or
        (len(hit_result['block_reasons']) > 0)):
        num_rejected_hits += 1
        print('| reject_reasons:', hit_result['reject_reasons'], '| block_reasons:', hit_result['block_reasons'])
#         continue
    elif hit_result['feedback'] is None:
        num_incomplete_hits += 1
        continue
    
    worker_ids.append(hit_result['worker_id'])
    num_valid_hits += 1
    if (hit_result['task_rating'] is not None) and (hit_result['task_rating'].isdigit()):
        task_ratings[int(hit_result['task_rating'])] += 1
    for qtype, qtype_accuracy in hit_result['accuracy'].items():
        accuracy_by_worker[qtype] = accuracy_by_worker.get(qtype, []) + [qtype_accuracy]
    
    hit_durations = []
    response_option_counts = {option: 0 for option in options}
    responses = []
    for prompt in hit_result['data']:
        qid = prompt['sample']['qid']
        if (split is not None) and (split not in qid):
            continue
        model_stance = debate_mode_to_option[prompt['sample']['debate_mode']]
        answer = prompt['sample']['eval_labels'][0]
        human_correct = (prompt['response'] == answer)
        assert answer in options, 'Answer must be in options.'
        
        # Calculate metrics
        if qid not in metrics:
            metrics[qid] = {
                option: {
                    'num': 0,
                    'num_correct': 0,
                    'num_correct_debate_mode': 0,
                    'num_incorrect_debate_mode': 0,
                    'num_correct_with_correct_debate_mode': 0,
                    'num_correct_with_incorrect_debate_mode': 0,
                    'num_debate_mode_responses': 0,
                }
                for option in ([None] if model_stance is None else options)
            }
            metrics[qid]['answer'] = answer
        prompt_metrics = metrics[qid][model_stance]
        prompt_metrics['num'] += 1
        prompt_metrics['num_correct'] += human_correct
        if model_stance == answer:
            prompt_metrics['num_correct_with_correct_debate_mode'] += human_correct
            prompt_metrics['num_correct_debate_mode'] += 1
        else:
            prompt_metrics['num_correct_with_incorrect_debate_mode'] += human_correct
            prompt_metrics['num_incorrect_debate_mode'] += 1
        prompt_metrics['num_debate_mode_responses'] += (prompt['response'] == model_stance)
        
        hit_durations.append(prompt['duration'] / 1000.)
        response_option_counts[prompt['response']] += 1
        responses.append(prompt['response'])
    duration = np.median(np.array(hit_durations))
    durations_by_worker.append(duration)
    durations += hit_durations
    response_options_array = np.array(list(response_option_counts.values()))
    response_options_array = response_options_array / response_options_array.sum()
    max_response_freq = response_options_array.max()
    max_response_freqs.append(max_response_freq)
    print('| Time:', round(duration, 1),
          '| Acc:', round(100 * hit_result['accuracy'][prompt_type]),
          '| Max Freq:', round(100 * max_response_freq, 1),
          '| Rate:', hit_result['task_rating'],
          '| Feedback:', hit_result['feedback'])

durations = np.array(durations)
durations_by_worker = np.array(durations_by_worker)
max_response_freqs = np.array(max_response_freqs)
durations.sort()
durations_by_worker.sort()
max_response_freqs.sort()
for qtype in accuracy_by_worker:
    accuracy_by_worker[qtype] = np.array(accuracy_by_worker[qtype])
    accuracy_by_worker[qtype].sort()
print('REJECTED:', num_rejected_hits)
print('INCOMPLETE:', num_incomplete_hits)
print('VALID:', num_valid_hits)
print('Median Question Duration:', np.median(durations))
print('Median Worker Duration:', np.median(durations_by_worker))
print('Median Worker Accuracy:', np.median(accuracy_by_worker[prompt_type]))
print('Median Max Response Freq:', np.median(max_response_freqs))
# pprint(hit_results[0]['data'][0])
pprint(hit_results[0]['worker_id'])

| Time: 9.5 | Acc: 25 | Max Freq: 40.0 | Rate: 6 | Feedback: Let successful workers skip the pretest questions.
| Time: 5.7 | Acc: 30 | Max Freq: 45.0 | Rate: 6 | Feedback: N/A
| Time: 7.7 | Acc: 40 | Max Freq: 50.0 | Rate: 6 | Feedback: No improvements.
| Time: 5.6 | Acc: 40 | Max Freq: 40.0 | Rate: 10 | Feedback: No comment
| Time: 12.8 | Acc: 30 | Max Freq: 50.0 | Rate: 7 | Feedback: Its already good
| Time: 18.7 | Acc: 65 | Max Freq: 40.0 | Rate: 5 | Feedback: It's very long for the pay amount. I'd say pay more.
| Time: 8.9 | Acc: 60 | Max Freq: 45.0 | Rate: 10 | Feedback: I would make the instructions more clear so that the worker knows what they are doing. 
| Time: 10.7 | Acc: 65 | Max Freq: 40.0 | Rate: 9 | Feedback: Some of the questions seem impossible to guess right without context. I suppose that's part of the design but it's pretty hard that way.
| Time: 3.0 | Acc: 30 | Max Freq: 35.0 | Rate: 10 | Feedback: Nothing everything went smooth! 
| Time: 8.7 | Acc: 35 | Max Freq: 

In [5]:
accuracy_by_sample = []
accuracy_by_sample_correct_debate_mode = []
accuracy_by_sample_incorrect_debate_mode = []
convinced_freqs = []
convinced_freqs_with_correct_debate_mode = []
convinced_freqs_with_incorrect_debate_mode = []
num_evals_by_sample = []
for qid, qid_metrics in metrics.items():
    answer = metrics[qid]['answer']
    for model_stance, prompt in qid_metrics.items():
        if not (model_stance in [None] + options):
            continue

        # Q-only stats
        prompt_metrics = metrics[qid][model_stance]
        num_evals_by_sample.append(prompt_metrics['num'])
        accuracy_by_sample.append(prompt_metrics['num_correct'] / prompt_metrics['num'])
        if model_stance is None:
            continue
        
        # Debater stats
        convinced_freq = prompt_metrics['num_debate_mode_responses'] / prompt_metrics['num']
        if model_stance == answer:
            convinced_freqs_with_correct_debate_mode.append(convinced_freq)
            accuracy_by_sample_correct_debate_mode.append(
                prompt_metrics['num_correct_with_correct_debate_mode'] /
                prompt_metrics['num_correct_debate_mode'])
        else:
            convinced_freqs_with_incorrect_debate_mode.append(convinced_freq)
            accuracy_by_sample_incorrect_debate_mode.append(
                prompt_metrics['num_correct_with_incorrect_debate_mode'] /
                prompt_metrics['num_incorrect_debate_mode'])
        convinced_freqs.append(convinced_freq)

worker_ids = set(worker_ids)
        
num_evals_by_sample = np.array(num_evals_by_sample)
print('Evals per sample:', num_evals_by_sample.mean())
print('Fraction insuffient evals:', (num_evals_by_sample < 5).mean())

convinced_freqs = np.array(convinced_freqs)
print('Convinced:', round(100 * convinced_freqs.mean(), 2), '%')
convinced_freqs_with_correct_debate_mode = np.array(convinced_freqs_with_correct_debate_mode)
print('- Correct debater:', round(100 * convinced_freqs_with_correct_debate_mode.mean(), 2), '%')
convinced_freqs_with_incorrect_debate_mode = np.array(convinced_freqs_with_incorrect_debate_mode)
print('- Incorrect debater:', round(100 * convinced_freqs_with_incorrect_debate_mode.mean(), 2), '%')

accuracy_by_sample = np.array(accuracy_by_sample)
print('Accuracy:', round(100 * accuracy_by_sample.mean(), 2), '%')
accuracy_by_sample_correct_debate_mode = np.array(accuracy_by_sample_correct_debate_mode)
print('- Correct debater:', round(100 * accuracy_by_sample_correct_debate_mode.mean(), 2), '%')
accuracy_by_sample_incorrect_debate_mode = np.array(accuracy_by_sample_incorrect_debate_mode)
print('- Incorrect debater:', round(100 * accuracy_by_sample_incorrect_debate_mode.mean(), 2), '%')

num_target_evals = 5
print('Extra Evals:', round(((100. * (num_evals_by_sample - num_target_evals).sum()) / num_evals_by_sample.sum()), 2), '%')
num_evals_by_sample.sort()
print('Evals per sample distribution:', num_evals_by_sample)

Evals per sample: 6.0
Fraction insuffient evals: 0.0
Convinced: nan %
- Correct debater: nan %
- Incorrect debater: nan %
Accuracy: 43.28 %
- Correct debater: nan %
- Incorrect debater: nan %
Extra Evals: 16.67 %
Evals per sample distribution: [5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]


/Users/ethanperez/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice.
/Users/ethanperez/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/ethanperez/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: RuntimeWarning: Mean of empty slice.
/Users/ethanperez/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: RuntimeWarning: Mean of empty slice.
/Users/ethanperez/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:51: RuntimeWarning: Mean of empty slice.
/Users/ethanperez/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: RuntimeWarning: Mean of empty slice.


In [6]:
def nps(task_ratings):
    num_ratings = sum(list(task_ratings.values()))
    print(num_ratings)
    if num_ratings == 0:
        return None

    nps_sum_ratings = 0
    sum_ratings = 0
    for score, num_raters in task_ratings.items():
        sum_ratings += num_raters * score
        if score >= 9:
            nps_sum_ratings += num_raters
        elif score <= 6:
            nps_sum_ratings -= num_raters
    return round(100 * (nps_sum_ratings / float(num_ratings)), 2), round((sum_ratings / float(num_ratings)), 2)

print('NPS, Mean:', nps(task_ratings))

30
NPS, Mean: (23.33, 8.03)


In [7]:
durations_by_worker

array([ 3.0275,  4.4515,  5.2565,  5.6405,  5.6475,  5.703 ,  6.156 ,
        6.2705,  6.6475,  6.8395,  7.2245,  7.317 ,  7.4135,  7.7045,
        7.7315,  7.917 ,  7.932 ,  8.148 ,  8.1855,  8.6555,  8.87  ,
        8.9845,  8.9915,  9.451 ,  9.4655,  9.6525, 10.696 , 11.7755,
       12.808 , 18.698 ])

In [11]:
accuracy_by_worker[prompt_type].mean()

0.4366666666666667